<a href="https://colab.research.google.com/github/umass-gis/workshops/blob/main/content/web-map/python/Sunlight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook for generating sunset times from the NOAA Solar Calculations
# Coordinate extremes from https://en.wikipedia.org/wiki/List_of_extreme_points_of_the_United_States
# Sunrise/sunset API from https://sunrise-sunset.org/api
# Cities data from https://simplemaps.com/data/us-cities
# ISO8601/UTC to Local Time code from https://medium.com/xster-tech/python-convert-iso8601-utc-to-local-time-a386652b0306

import csv
import pandas as pd
import requests
import json
import pytz, dateutil.parser
from datetime import *

# If this is the first session run this code to mount to Google Drive
#from google.colab import drive
#drive.mount('/drive')

# Optional code to print a formatted JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# Define the CSV to be queried
file = ('/drive/MyDrive/Colab Notebooks/Sunlight/uscities_1000.csv')

# Define columns and rows for the output CSV
cols = ['city', 'state_id', 'state_name', 'lat', 'lng', 'population', 'density', 'timezone', 'sunrise_long', 'sunset_long', 'sunrise_short', 'sunset_short', 'day_length', 'night_length']
rows = []

# Create a place to store multiple dataframes for each processed row
dfs = []

# Start the counter
record_count = 0


In [ ]:
# Open the CSV and make sure to close it when done processing
with open(file, 'r', encoding='windows-1252') as f:

    # Read the CSV and load values into dictionary
    reader = csv.DictReader(f)

    # Process one row at a time
    for row in reader:

      # Increase the count each time a file is processed
      record_count = record_count + 1

      # Print the number of records processed
      print(record_count)
      
      # Retrieve important values from the row
      city = row['city']
      state_id = row['state_id']
      state_name = row['state_name']
      lat = row['lat']
      lng = row['lng']
      population = row['population']
      density = row['density']
      timezone = row['timezone']
    
      # Set Sunrise/Sunset API Parameters
      # lat (float): Latitude in decimal degrees. Required.
      # lng (float): Longitude in decimal degrees. Required.
      # date (string): Date in YYYY-MM-DD format. Also accepts other date formats and even relative date formats. If not present, date defaults to current date. Optional.
      # callback (string): Callback function name for JSONP response. Optional.
      # formatted (integer): 0 or 1 (1 is default). Time values in response will be expressed following ISO 8601 and day_length will be expressed in seconds. Optional.

      parameters = {
          'lat': float(lat),
          'lng': float(lng),
          'date': "2022-12-21",
          'formatted': 0
          }
      
      # Call the API
      sunlight = requests.get('https://api.sunrise-sunset.org/json?', params=parameters)

      # Print the formatted JSON of this query <-- useful for testing
      # jprint(sunlight.json())

      # Filter out just the 'results' key
      results = sunlight.json()['results']

      # Locate individual values from results
      sunrise = results['sunrise']
      sunset = results['sunset']
      day_length = results['day_length']

      # Convert the sunrise/sunset times to local timezones
      sunrise_utc = dateutil.parser.parse(sunrise)
      sunrise_local = sunrise_utc.astimezone(pytz.timezone(timezone))
      sunset_utc = dateutil.parser.parse(sunset)
      sunset_local = sunset_utc.astimezone(pytz.timezone(timezone))

      # Format the sunrise/sunset times to simple hours and minutes
      sunrise_time = datetime.strptime(str(sunrise_local), "%Y-%m-%d %H:%M:%S%z")
      sunrise_format = sunrise_time.strftime("%-I:%M %p")
      sunset_time = datetime.strptime(str(sunset_local), "%Y-%m-%d %H:%M:%S%z")
      sunset_format = sunset_time.strftime("%-I:%M %p")

      # Convert day length from seconds to decimal hours
      day_length_hours = (day_length / 3600)

      # Calculate night length
      night_length_hours = (24 - day_length_hours)

      # Stage the information retrieved from this file into a temporary dataframe
      this_df = pd.DataFrame([{'city': city,
                               'state_id': state_id,
                               'state_name': state_name,
                               'lat': lat,
                               'lng': lng,
                               'population': population,
                               'density': density,
                               'timezone': timezone, 
                               'sunrise_long': sunrise_local,
                               'sunset_long': sunset_local,
                               'sunrise_short': sunrise_format,
                               'sunset_short': sunset_format,
                               'day_length': day_length_hours,
                               'night_length': night_length_hours,
                               }], columns=cols)

      # Append this dataframe to the dfs list
      dfs.append(this_df)

# Concatenate all the dfs into a single dataframe
df = pd.concat(dfs)

# Write the dataframe to csv
df.to_csv('/drive/MyDrive/Colab Notebooks/Sunlight/uscities_1000_sunlight.csv', index=False)



In [ ]:
### Testing a point in NYC ###

import requests
import json

# Optional code to print a formatted JSON object
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# Input data
lat = 40.7127281
lng = -74.0060152
date = "2021-12-21"
formatted = 0

# Set API Parameters
# lat (float): Latitude in decimal degrees. Required.
# lng (float): Longitude in decimal degrees. Required.
# date (string): Date in YYYY-MM-DD format. Also accepts other date formats and even relative date formats. If not present, date defaults to current date. Optional.
# callback (string): Callback function name for JSONP response. Optional.
# formatted (integer): 0 or 1 (1 is default). Time values in response will be expressed following ISO 8601 and day_length will be expressed in seconds. Optional.

parameters = {
    'lat': float(lat),
    'lng': float(lng),
    'date': str(date),
    'formatted': 0
    }

print(parameters)

In [ ]:
# Call the API
sunlight = requests.get('https://api.sunrise-sunset.org/json?', params=parameters)

# Print the formatted JSON of this query <-- useful for testing
jprint(sunlight.json())


In [ ]:
# Filter out just the 'results' key
results = sunlight.json()['results']

# Locate individual values from results
sunrise = results['sunrise']
sunset = results['sunset']
day_length = results['day_length']

print (sunrise, sunset, day_length)


In [ ]:
# Parse date result and format as local time

import pytz, dateutil.parser

utc_sunrise = dateutil.parser.parse(sunrise)
local_sunrise = utc_sunrise.astimezone(pytz.timezone("America/New_York"))

utc_sunset = dateutil.parser.parse(sunset)
local_sunset = utc_sunset.astimezone(pytz.timezone("America/New_York"))

print(local_sunrise, local_sunset)

In [ ]:
# Calculate daylight in hours

day_length_hours = (day_length / 3600)

print(day_length_hours)

In [ ]:
# Format time as human readable

from datetime import *

time = datetime.strptime(str(local_sunrise), "%Y-%m-%d %H:%M:%S%z")
sunrise_format = time.strftime("%-I:%M %p")

time = datetime.strptime(str(local_sunset), "%Y-%m-%d %H:%M:%S%z")
sunset_format = time.strftime("%-I:%M %p")

print(sunrise_format, sunset_format)
